In [1]:
import time
ini=time.time()
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
def un_dumm(pred_dumm):
    pred=pd.concat([pred_dumm.select_dtypes(exclude='bool'),pd.from_dummies(pred_dumm.select_dtypes(include='bool'),sep='~')],axis=1)
    return pred

In [3]:
pred_c=un_dumm(pd.read_csv('~/git/BD/temp/pred_c.csv'))
pred_c.head(1)

oc  oc_pred                 date MAP_k~kg_ad MAP_k~fob_ad MOP_k~kg_ad  \
0   1        1  1491004800000000000  16605218.0    5946978.0  23312647.0   

  MOP_k~fob_ad SAM_k~kg_ad SAM_k~fob_ad UREIA_k~kg_ad UREIA_k~fob_ad  md_c  \
0    5091609.0  13094598.0    1601334.0     5002341.0      1511247.0     1   

   l_min_y  indv  l_max_y  ad_col CO_MES   nick  
0       20     0        0       1      4  MAP_k

In [4]:
pred_r=un_dumm(pd.read_csv('~/git/BD/temp/pred_r.csv'))
pred_r.head(1)

KG_LIQUIDO  KG_LIQUIDO_pred                 date MAP_k~kg_ad MAP_k~fob_ad  \
0    432000.0     1.801647e+07  1491004800000000000  16605218.0    5946978.0   

  MOP_k~kg_ad MOP_k~fob_ad SAM_k~kg_ad SAM_k~fob_ad UREIA_k~kg_ad  \
0  23312647.0    5091609.0  13094598.0    1601334.0     5002341.0   

  UREIA_k~fob_ad  md_r  l_min_y  indv  l_max_y  ad_col CO_MES   nick  
0      1511247.0    34       20     0        0       1      4  MAP_k

In [5]:
pred=pred_c.merge(pred_r,how='outer').fillna(0)

pred.date=pred.date.astype('datetime64[ns]')

pred['CO_ANO']=pred.date.dt.year

pred.head(1)

oc  oc_pred       date MAP_k~kg_ad MAP_k~fob_ad MOP_k~kg_ad MOP_k~fob_ad  \
0   1        1 2017-04-01  16605218.0    5946978.0  23312647.0    5091609.0   

  SAM_k~kg_ad SAM_k~fob_ad UREIA_k~kg_ad  ... l_min_y  indv  l_max_y  ad_col  \
0  13094598.0    1601334.0     5002341.0  ...      20     0        0       1   

   CO_MES   nick KG_LIQUIDO KG_LIQUIDO_pred  md_r  CO_ANO  
0       4  MAP_k   432000.0    1.801647e+07  34.0    2017  

[1 rows x 22 columns]

In [6]:
scores=pd.DataFrame()
for mes in pred.CO_MES.unique():
    for n in pred.nick.unique():
        for mdc in pred.md_c.unique():
            for mdr in pred.md_r.unique():
                for ad in pred.ad_col.unique():
                    pred_t=pred[(pred.CO_MES==mes)&(pred.nick==n)&(pred.md_c==mdc)&(pred.md_r==mdr)&(pred.ad_col==ad)]
                    ac=round(metrics.accuracy_score(pred_t.oc,pred_t.oc_pred),2)
                    pr=round(metrics.precision_score(pred_t.oc,pred_t.oc_pred),2)
                    rc=round(metrics.recall_score(pred_t.oc,pred_t.oc_pred),2)
                    pred_t=pred_t[(pred_t.oc==1)&(pred_t.oc_pred==1)]
                    if len(pred_t)>0:
                        r2=metrics.r2_score(pred_t.KG_LIQUIDO,pred_t.KG_LIQUIDO_pred)
                    else:
                        r2=np.nan
                    q=pd.DataFrame({'CO_MES':[mes],'nick':[n],'md_c':[mdc],'md_r':[mdr],'accuracy':[ac],
                                    'precision':[pr],'recall':[rc],'r2':[r2],'ad_col':[ad]
                                   })
                    scores=pd.concat([q,scores])

In [7]:
scores.CO_MES=scores.CO_MES.astype(int)

In [8]:
scores_g=scores.groupby(by=['CO_MES','nick']).r2.max().reset_index().merge(scores)
scores_g.groupby('md_r').CO_MES.count()

md_r
1.0     11
2.0      9
13.0    12
21.0     6
34.0    10
Name: CO_MES, dtype: int64

In [9]:
i=pd.read_csv('~/git/BD/temp/imp_fert_quanti.csv').sort_values(by=['CO_ANO','CO_MES'])
i=round(((i.groupby(by=['CO_MES','nick']).KG_LIQUIDO.sum())/10**9),2).reset_index().rename(columns={'KG_LIQUIDO':'mt_sum'})
scores_g_f=scores_g.merge(i)
scores_g_f.head()

CO_MES     nick        r2  md_c  md_r  accuracy  precision  recall  ad_col  \
0       1    MAP_k  0.132413     1  13.0       1.0        1.0     1.0       1   
1       1    MOP_k  0.021345     1   2.0       1.0        1.0     1.0       1   
2       1    SAM_k  0.042862     1  13.0       1.0        1.0     1.0       1   
3       1  UREIA_k  0.036772     1  34.0       1.0        1.0     1.0       1   
4       2    MAP_k -0.146718     1   1.0       1.0        1.0     1.0       0   

   mt_sum  
0    3.25  
1   10.90  
2    5.67  
3    8.41  
4    3.21

In [10]:
scores_g_f.r2=round(scores_g_f.r2,2)

r2_pos_class_pos=scores_g_f[(scores_g_f.r2>0)&(scores_g_f.accuracy>.95)&(scores_g_f.precision>.95)&(scores_g_f.recall>.95)].sort_values('mt_sum',ascending=False).reset_index(drop=True)

r2_pos_class_out=scores_g_f[(scores_g_f.r2>0)&((scores_g_f.accuracy<=.95)|(scores_g_f.precision<=.95)|(scores_g_f.recall<=.95))].sort_values('mt_sum',ascending=False).reset_index(drop=True)

r2_out_class_pos=scores_g_f[(scores_g_f.r2<=0)&(scores_g_f.accuracy>.95)&(scores_g_f.precision>.95)&(scores_g_f.recall>.95)].sort_values('mt_sum',ascending=False).reset_index(drop=True)

r2_out_class_out=scores_g_f[(scores_g_f.r2<=0)&((scores_g_f.accuracy<=.95)|(scores_g_f.precision<=.95)|(scores_g_f.recall<=.95))].sort_values('mt_sum',ascending=False).reset_index(drop=True)

In [11]:
len(r2_pos_class_pos[['CO_MES','nick']].drop_duplicates())

33

In [12]:
len(r2_pos_class_out[['CO_MES','nick']].drop_duplicates())

0

In [13]:
len(r2_out_class_pos[['CO_MES','nick']].drop_duplicates())

15

In [14]:
len(r2_out_class_out[['CO_MES','nick']].drop_duplicates())

0